In [4]:
# AiiDA環境をロード
from aiida import load_profile

# プロファイルをロード
load_profile("test_aiida")

Profile<uuid='efe3b646acbe4b46b34848999a0e7c25' name='test_aiida'>

In [5]:
# Int ノードをインポート
from aiida.orm import Int
node = Int(2)
node.store()

<Int: uuid: e87f342c-92dd-4b42-b069-214c43fc4cd0 (pk: 2) value: 2>